In [23]:
#Memory efficient implementation of load data frames
""" 
    Takes in tickers as inputs, gradually yields dataframes. 
    Concat all data frames using lazy loading. 
"""

#takes in tickers as inputs, gradually passes data frames. 
#uses a generator ins
import pandas as pd
import numpy as np
import os
from utils.file_utils import load_fin_data_yield, date_today_string
from utils.financial_formula_utils import create_prior_year_BS_item, compute_prior_year_BS_item_and_average, compute_rate_of_change, compute_ratio, compute_margins
from src.constants import roc_is_metrics, roc_bs_metrics, financial_ratio_tuples
import yfinance as yf


ticker_list = pd.read_csv('ASX_tickers_final.csv')[:1666]
tickers = ticker_list['Ticker']

directory_statements = "./data/AU/financial_statements"

# date_today = date_today_string()
date_today = date_today_string()

df_IS = pd.concat(load_fin_data_yield(tickers, "IS",date_today,directory_statements), axis=0)
df_BS = pd.concat(load_fin_data_yield(tickers, "BS",date_today,directory_statements), axis=0)
df_CF = pd.concat(load_fin_data_yield(tickers, "CF",date_today,directory_statements), axis=0)


In [24]:
#Merges Income Statement, Balance Sheet and Company Information
merge_is_df = df_IS.merge(df_BS, on=['ticker','FY_End'], how='outer')
merge_is_df = merge_is_df.sort_values(by = ['ticker','FY_End'])
merge_is_df = merge_is_df.merge(ticker_list,left_on='ticker', right_on="Ticker",how='inner')

print(df_IS.shape)
print(df_BS.shape)
print(ticker_list.shape)
print(merge_is_df.shape)

(7153, 82)
(7282, 139)
(1666, 9)
(7368, 228)


In [ ]:
tickers = ["AAPL", "GOOGL", "MSFT"]
df = yf.download(tickers, start="2023-01-01", end="2023-12-31")["Close"]

In [41]:
tickers

0    CBA.AX
1    BHP.AX
2    RIO.AX
3    CSL.AX
4    NAB.AX
Name: Ticker, dtype: object

In [40]:
tickers = tickers[:5]
data = yf.download(tickers, period="3y")
print(data)


AttributeError: 'Series' object has no attribute 'split'

In [39]:

import time
# Define the tickers
# tickers_to_show = ["CBA.AX", "ALL.AX"]


all_data = {}
batch_size = 10


tickers = tickers[:5]

for i in range(0, len(tickers),batch_size):
    batch = tickers[i:i+batch_size]
    print(f"Downloading batch {i//batch_size + 1} of {len(tickers)//batch_size + 1}...")

    try:
        data = yf.download(batch, period="3y")  # Avoid multithreading
        print(data)
        # for ticker in batch:
        #     all_data[ticker] = data["Close"][ticker]
        # all_data.update({ticker: data["Close"] for ticker in batch})  # Store only Close prices
    except Exception as e:
        print(f"Error in batch {i//batch_size + 1}: {e}")

    time.sleep(10)  # Sleep to avoid rate limits


# # Download historical price data
# data = yf.download(tickers_to_show, period="3y")

# # Display the first few rows
# # print(data.head())
# px_close = data["Close"]
# px_close.columns = [ticker for ticker in px_close.columns]
# px_close

Error in batch 1: 'Series' object has no attribute 'split'


In [38]:
all_data

{}

In [33]:
data

Price           Close                   High                    Low  \
Ticker         ALL.AX      CBA.AX     ALL.AX      CBA.AX     ALL.AX   
Date                                                                  
2022-03-14  33.828396   90.843323  34.009959   90.914574  33.111693   
2022-03-15  34.048187   92.428612  34.439985   92.446428  33.589498   
2022-03-16  35.500710   93.301430  35.500710   93.328147  34.459100   
2022-03-17  36.217407   94.539391  36.666539   95.296417  35.634488   
2022-03-18  36.160069   94.664085  36.427637   94.664085  35.682266   
...               ...         ...        ...         ...        ...   
2025-03-06  72.269997  153.589996  73.089996  156.880005  71.379997   
2025-03-07  69.760002  148.500000  72.269997  150.960007  69.360001   
2025-03-10  69.389999  148.139999  69.489998  149.580002  67.870003   
2025-03-11  68.190002  146.919998  68.690002  148.130005  66.500000   
2025-03-12  67.099998  144.800003  68.000000  145.839996  66.250000   

Price                        Open               Volume           
Ticker          CBA.AX     ALL.AX      CBA.AX   ALL.AX   CBA.AX  
Date                                                             
2022-03-14   89.151142  33.216806   89.355988  1322468  3220955  
2022-03-15   91.110497  33.637277   91.333152  1426454  3188234  
2022-03-16   92.384091  34.592885   92.900646  2046940  2869840  
2022-03-17   93.764548  36.322521   95.296417  2615381  3996962  
2022-03-18   93.604244  36.217406   94.174241  3185116  6310578  
...                ...        ...         ...      ...      ...  
2025-03-06  153.589996  72.620003  156.440002  1064651  2374229  
2025-03-07  147.220001  71.110001  150.880005  1166600  2627604  
2025-03-10  147.410004  69.000000  149.199997  1199622  1456710  
2025-03-11  144.399994  68.480003  145.690002  1730601  2523185  
2025-03-12  142.979996  67.510002  145.750000  1362803  2921264  

[758 rows x 10 columns]

In [36]:
all_data["ALL.AX"] = data["Close"]["ALL.AX"]
all_data

{'ALL.AX': Date
 2022-03-14    33.828396
 2022-03-15    34.048187
 2022-03-16    35.500710
 2022-03-17    36.217407
 2022-03-18    36.160069
                 ...    
 2025-03-06    72.269997
 2025-03-07    69.760002
 2025-03-10    69.389999
 2025-03-11    68.190002
 2025-03-12    67.099998
 Name: ALL.AX, Length: 758, dtype: float64}

In [35]:
data["Close"]["ALL.AX"]

Date
2022-03-14    33.828396
2022-03-15    34.048187
2022-03-16    35.500710
2022-03-17    36.217407
2022-03-18    36.160069
                ...    
2025-03-06    72.269997
2025-03-07    69.760002
2025-03-10    69.389999
2025-03-11    68.190002
2025-03-12    67.099998
Name: ALL.AX, Length: 758, dtype: float64

In [34]:
data["Close"]

Ticker,ALL.AX,CBA.AX
Date,,
2022-03-14,33.828396,90.843323
2022-03-15,34.048187,92.428612
2022-03-16,35.500710,93.301430
2022-03-17,36.217407,94.539391
2022-03-18,36.160069,94.664085
...,...,...
2025-03-06,72.269997,153.589996
2025-03-07,69.760002,148.500000
2025-03-10,69.389999,148.139999


In [27]:
all_data

{}

In [20]:
# merge_is_df[merge_is_df["ticker"].isin(tickers_to_show)]
merge_is_df = merge_is_df[merge_is_df["ticker"].isin(tickers_to_show)]
merge_is_df["FY"] = merge_is_df["FY_End"].apply(lambda x:(x[:4]))
merge_is_df

merge_is_df = merge_is_df[["ticker", "FY_End","FY", "Ordinary Shares Number","Common Stock Equity"]]
merge_is_df = compute_ratio(merge_is_df, "Common Stock Equity", "Ordinary Shares Number", "BPS")
merge_is_df

,ticker,FY_End,FY,Ordinary Shares Number,Common Stock Equity,BPS
0,ALL.AX,2020-09-30,2020,NaN,NaN,NaN
1,ALL.AX,2021-09-30,2021,6.411028e+08,3.878900e+09,6.050356
2,ALL.AX,2022-09-30,2022,6.597926e+08,6.022700e+09,9.128171
3,ALL.AX,2023-09-30,2023,6.485601e+08,6.726100e+09,10.370820
4,ALL.AX,2024-09-30,2024,6.293817e+08,6.279900e+09,9.977887
15,CBA.AX,2020-06-30,2020,NaN,NaN,NaN
16,CBA.AX,2021-06-30,2021,1.772431e+09,7.868300e+10,44.392692
17,CBA.AX,2022-06-30,2022,1.699938e+09,7.283300e+10,42.844504
18,CBA.AX,2023-06-30,2023,1.674519e+09,7.162800e+10,42.775259
19,CBA.AX,2024-06-30,2024,1.672070e+09,7.308800e+10,43.711088


In [22]:
def compute_pb_ratio(price_df, bps_df):
    # Ensure FY_End is datetime
    bps_df['FY_End'] = pd.to_datetime(bps_df['FY_End'])
    print(bps_df)
    # Create an empty DataFrame for P/B ratios
    pb_df = pd.DataFrame(index=price_df.index, columns=price_df.columns)
    
    for ticker in price_df.columns:
        # Filter BPS data for the specific ticker
        ticker_bps = bps_df[bps_df['ticker'] == ticker]

        if ticker_bps.empty:
            continue  # Skip if no BPS data for this ticker

        # Merge price data with the nearest available BPS based on FY_End
        for _, row in ticker_bps.iterrows():
            # Select all price data after the FY_End
            relevant_dates = price_df.index[price_df.index >= row['FY_End']]
            pb_df.loc[relevant_dates, ticker] = price_df.loc[relevant_dates, ticker] / row['BPS']
    
    return pb_df

# Example usage
pb_ratio_df = compute_pb_ratio(px_close, merge_is_df)
pb_ratio_df

    ticker     FY_End    FY  Ordinary Shares Number  Common Stock Equity  \
0   ALL.AX 2020-09-30  2020                     NaN                  NaN   
1   ALL.AX 2021-09-30  2021            6.411028e+08         3.878900e+09   
2   ALL.AX 2022-09-30  2022            6.597926e+08         6.022700e+09   
3   ALL.AX 2023-09-30  2023            6.485601e+08         6.726100e+09   
4   ALL.AX 2024-09-30  2024            6.293817e+08         6.279900e+09   
15  CBA.AX 2020-06-30  2020                     NaN                  NaN   
16  CBA.AX 2021-06-30  2021            1.772431e+09         7.868300e+10   
17  CBA.AX 2022-06-30  2022            1.699938e+09         7.283300e+10   
18  CBA.AX 2023-06-30  2023            1.674519e+09         7.162800e+10   
19  CBA.AX 2024-06-30  2024            1.672070e+09         7.308800e+10   

          BPS  
0         NaN  
1    6.050356  
2    9.128171  
3   10.370820  
4    9.977887  
15        NaN  
16  44.392692  
17  42.844504  
18  42.775259  
19 

,ALL.AX,CBA.AX
Date,,
2022-03-14,5.591142,2.046358
2022-03-15,5.627469,2.082068
2022-03-16,5.867541,2.101729
2022-03-17,5.985996,2.129616
2022-03-18,5.976519,2.132425
...,...,...
2025-03-06,7.243016,3.513754
2025-03-07,6.99146,3.397307
2025-03-10,6.954378,3.389071
